In [1]:
from Crypto import Random
from Crypto.PublicKey import RSA
import base64

In [2]:
def toBase64(string):
    return base64.b64encode(string)

In [3]:
ori_key = RSA.generate(256*5,Random.new().read)

In [4]:
ori_key

RsaKey(n=11777440732895235409886786895856667944021108676627204811862057689005038678119703469213187705646687452731590545040438188690540606493337652045541342579961682704758237969889114788707269019941250998394600738076260331883915554332362060732842073143777569611226207953471729309918421951620949897438688213710571031819400032184815423773266588749179700369347944166685732401534157120154586856463977, e=65537, d=2843050507572349539986784748857903401260142359561479679052499865250228335248680420749915865803796616353207016155946295438892498360130129914193307754188989444598130884883817393453214970626982640514574304806972590255441225282778023288594263140341389724824205253485636612330992196616551752821311339661386386960254125083384871133872715962314109611134982183957551636236041853682380983171977, p=3279678725033217867388769301660577656893259718918467521272663395059085857715432354058208480867520115203116960794174593087142045316415921837358207942907861173899454549906671376776548552278589773, q=359103

**Writing public key to file**

In [5]:
f = open('Public.txt','w')
f.write(ori_key.public_key().export_key().decode())
f.close()

**Testing keys**

In [6]:
from Crypto.Cipher import PKCS1_OAEP

pub_key = PKCS1_OAEP.new(ori_key.public_key())
encrypted = pub_key.encrypt(b'hey whatssup')


In [7]:
str.encode('hey')

b'hey'

In [8]:
prikey = PKCS1_OAEP.new(ori_key)
prikey.decrypt(encrypted)

b'hey whatssup'

**Sharding key**

In [9]:
class Shard(RSA.RsaKey):
    def __init__(self,x,n,d,e,q,p,u):
        RSA.RsaKey.__init__(self,n=n,d=d,e=e,q=q,p=p,u=u)
        self.__x = x 
    def get_key():
        return super().export_key().decode()


For 2 shards to be able to form the key, we must have all of the shards in a straight line that follows the rule of y=ax+b.
In here, 'x' and 'b' will randomized; and 'a' will be the hidden values like d,q,p and u. The shards will be given 'x' and 'y'.
2 Shards will be needed to compute 'a', which will be the needed attributes of the private key

In [10]:
#Determining the equation 
import random
b = random.randint(1,1000000)
#Getting a list of 5 random numbers from 1 to 100
li = range(1,1000001)
x = random.sample(li,5)
#Printing equations:
print("The equations are:")
for i in range(1,6):
    print("x{:d}: a*{:d} + {:d}".format(i,x[i-1],b))


The equations are:
x1: a*895988 + 913391
x2: a*825207 + 913391
x3: a*811050 + 913391
x4: a*156031 + 913391
x5: a*252065 + 913391


In [11]:
def sharding_linear(nums,private_key):
    if isinstance(private_key,RSA.RsaKey):
        b = random.randint(1,1000000)
        li = range(1,1000001)
        x = random.sample(li,nums)
        list = []
        for i in range(0,nums):
            n = private_key.n
            d = private_key.d * x[i] + b
            e = private_key.e
            q = private_key.q * x[i] + b 
            p = private_key.p * x[i] + b 
            u = private_key.u * x[i] + b 
            list.append(Shard(x=x[i],n=n,d=d,e=e,q=q,p=p,u=u))
        return list 
    else:
        print("Key is not RSA key")

    
    

In [12]:
shards = sharding_linear(5,ori_key)

In [13]:
for x in shards:
    print(x)

Private RSA key at 0x244A7C0B908
Private RSA key at 0x244A7C0BE08
Private RSA key at 0x244A7C0B988
Private RSA key at 0x244A7CAD0C8
Private RSA key at 0x244A7CAD4C8


In [134]:
shards[0].__dict__

{'_n': 13685199152547765789497065624398310214666527124503818174596319337243682270660782077179612735010011872526682561871393690150701302126678845754288719257315058474661195483786573772711115507135235240398519967635716600591713510122979857036118201804067349120727854122639633724082765344885155214267534052253219653571422833606855208430482943590583155639673454331573180383043544163503225170014847,
 '_d': 1018732249685876313536518868120285210320577865406827918466578060551279316612883762627186542190929245657656870163656637288466803859677642822451711982559561136232132244778757557213444378661271143858159441503205617278425207715259436552535575016742001287315830133059582331724380490468758147977598226135431726047833300249924912404312138182849442084465559515147076129970398208680757497508380512531,
 '_e': 65537,
 '_q': 6415922381641696922432718781828627877226671412879403810821226598341746083801283871545779830703331323511617447919645878559342729900872792046305696545781137370602845276451156910200160809

Since shards are not keys, we cannot export them as keys. Instead, we will export them as dictionaries

In [15]:
for i in range(len(shards)):
    f = open('Shard[{:d}].txt'.format(i+1),'w')
    f.write(str(shards[i].__dict__))
    f.close()

In [17]:
#Opening shards 
f2 = open('Shard[1].txt','r')
dict1 = eval(f2.read())
shardtest1 = object.__new__(RSA.RsaKey)
shardtest1.__dict__ = dict1
f2.close()

In [18]:
shardtest1

RsaKey(n=11777440732895235409886786895856667944021108676627204811862057689005038678119703469213187705646687452731590545040438188690540606493337652045541342579961682704758237969889114788707269019941250998394600738076260331883915554332362060732842073143777569611226207953471729309918421951620949897438688213710571031819400032184815423773266588749179700369347944166685732401534157120154586856463977, e=65537, d=62891120278007944174047665429485681139275609135859491980320349519200301004036059587408888867445784950349292404385688001403740956224438603831870160830414635503955253304514924560578568365239482990822898196635040669040615344480332653166993694927491882102836244412355767501373878381354741324160228144649528265947781500969556734352398349802350418707916940891324999745177481845307949728748214957, p=72549773076459812444506965722033638348135798242195420038072586962102038258523079104121629805270412468408150289727936173680669184444436606964200917905064797027829834098485477525674030524954685280273, 

In [19]:
f3 = open('Shard[3].txt','r')
dict2 = eval(f3.read())
shardtest2 = object.__new__(RSA.RsaKey)
shardtest2.__dict__ = dict2
f3.close()

In [20]:
shardtest2

RsaKey(n=11777440732895235409886786895856667944021108676627204811862057689005038678119703469213187705646687452731590545040438188690540606493337652045541342579961682704758237969889114788707269019941250998394600738076260331883915554332362060732842073143777569611226207953471729309918421951620949897438688213710571031819400032184815423773266588749179700369347944166685732401534157120154586856463977, e=65537, d=1420066768875790154679379153852787596185224726750284800450896000194240800488357637319113226062740960512414312878685147269886097328406318200450672710216595770713986601297963298403765986033559678162703175785120318188919571292818946518350082713179699729483267809447780174583370299311411585361458337113446902263616452175524659128044681277868387667336978846118405594128631837370251430141622187539, p=16381568873306668929284142121785369521086436172234285867979188212078776178127081602314723794830750197824592813961998849773173187889606395507765392107792188541675168547692335659719879067319759708581

**Combining shards to find key**

In [21]:
def combine_linear_shards(shard1,shard2):
    #Solving linear equation
    d_difference = shard1.d-shard2.d
    q_difference = shard1.q - shard2.q
    p_difference = shard1.p - shard2.p
    u_difference = shard1.u - shard2.u
    x_difference = shard1._Shard__x-shard2._Shard__x
    d = d_difference // x_difference
    q = q_difference // x_difference
    p = p_difference // x_difference
    u = u_difference // x_difference
    
    return RSA.RsaKey(d=d,n=shard1.n,e=shard1.e,q=q,p=p,u=u)
    
    

In [22]:
combined = combine_linear_shards(shardtest1,shardtest2)

**Testing shards and keys**

In [23]:
#Try out shards
shard1 = PKCS1_OAEP.new(shards[0])
shard1.decrypt(encrypted)

ValueError: Fault detected in RSA decryption

In [24]:
#Try out combined key
combined_key = PKCS1_OAEP.new(combined)
combined_key.decrypt(encrypted)

b'hey whatssup'